<a href="https://colab.research.google.com/github/venkataravuri/ai-ml-models/blob/main/rag_add_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>